# Построение модели машинного обучения для поиска товара по текстовому описанию

# Разработка сервиса поиска кода товара по текстовому описанию

# Создание модели машинного обучения для сопоставления кодов с текстовым описанием товара

In [26]:
import pandas as pd
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import pymorphy3
import langid

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\223hy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\223hy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Импорт данных

In [27]:
df = pd.read_excel('C:\\Users\\223hy\\OneDrive\\Рабочий стол\\matcher\\sku_matcher\\mdr.xlsx')

In [28]:
df['ITEM'] = df['ITEM'].astype(str)
df['DESC'] = df['DESC'].astype(str)
df = df.dropna()

In [29]:
df

,ITEM,DESC
0,MDREX15LPLI.AE,"Наушники SONY MDR-EX15LP/LI, синие"
1,MDREX15LPPI.AE,"Наушники SONY MDR-EX15LP/P, розовые"
2,MDREX250APR.E,Наушники SONY MDR-EX250AP красные
3,MDRXB550APB.E,Наушники SONY MDR-XB550AP черный
4,MDRZX310L.AE,Наушники SONY MDR-ZX310/L синие
...,...,...
57119,MDRZX610APW.CE7,MDR-ZX610AP/W
57120,MDRZX660APB.E,MDR-ZX660AP/B
57121,MDRZX660APD.E,MDR-ZX660AP/D
57122,MDRZX660APL.E,MDR-ZX660AP/L


## Подготовка текста

In [30]:
# Drop rows with 'no' value in ITEM column (not relevant)
df = df[df['ITEM'] != 'no']

# Remove all text after last dot in ITEM column (region)
df['ITEM'] = df['ITEM'].apply(lambda x: x.rsplit('.', 1)[0] if '.' in x else x)

# Drop duplicates
df = df.drop_duplicates()   

# Reset the index
df = df.reset_index(drop=True)

In [31]:
df

,ITEM,DESC
0,MDREX15LPLI,"Наушники SONY MDR-EX15LP/LI, синие"
1,MDREX15LPPI,"Наушники SONY MDR-EX15LP/P, розовые"
2,MDREX250APR,Наушники SONY MDR-EX250AP красные
3,MDRXB550APB,Наушники SONY MDR-XB550AP черный
4,MDRZX310L,Наушники SONY MDR-ZX310/L синие
...,...,...
24512,MDRZX310APL,MDRZX310APL.CE7_RU
24513,MDREX450APH,MDR-EX450AP/B
24514,MDREX450APH,MDR-EX450AP/L
24515,MDREX650APT,MDR-EX650AP/T


In [32]:

def clean_text(text):
    # удаление лишних символов и приведение к нижнему регистру
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    
    # токенизация
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    
    # удаление стоп-слов
    stop_words = set(stopwords.words('english') + stopwords.words('russian'))
    tokens = [token for token in tokens if not token in stop_words]
    
    # объединение токенов в строку
    text = ' '.join(tokens)
    return text


# применение функции очистки текста к колонке 'DESC'
df['DESC'] = df['DESC'].apply(clean_text)

In [33]:
df

,ITEM,DESC
0,MDREX15LPLI,наушники sony mdrex15lpli синие
1,MDREX15LPPI,наушники sony mdrex15lpp розовые
2,MDREX250APR,наушники sony mdrex250ap красные
3,MDRXB550APB,наушники sony mdrxb550ap черный
4,MDRZX310L,наушники sony mdrzx310l синие
...,...,...
24512,MDRZX310APL,mdrzx310aplce7_ru
24513,MDREX450APH,mdrex450apb
24514,MDREX450APH,mdrex450apl
24515,MDREX650APT,mdrex650apt


## Лемматизация

In [34]:
def lemmatize(text):
    lemmatizer_en = WordNetLemmatizer()  # Создание объекта WordNetLemmatizer для английской лемматизации
    lemmatizer_ru = pymorphy3.MorphAnalyzer()  # Создание объекта MorphAnalyzer для русской лемматизации
    
    # Токенизация текста
    tokens = nltk.word_tokenize(text)
    
    # Лемматизация каждого слова в зависимости от языка
    lemmas = []
    for token in tokens:
        if langid.classify(token)[0] == 'en':
            lemma = lemmatizer_en.lemmatize(token)
        else:
            lemma = lemmatizer_ru.parse(token)[0].normal_form
        lemmas.append(lemma)
    
    # Объединение слов в предложение
    lemmatized_text = ' '.join(lemmas)
    
    return lemmatized_text


In [35]:
df['DESC'] = df['DESC'].apply(lemmatize)

In [36]:
df.head(15)

,ITEM,DESC
0,MDREX15LPLI,наушник sony mdrex15lpli синий
1,MDREX15LPPI,наушник sony mdrex15lpp розовый
2,MDREX250APR,наушник sony mdrex250ap красный
3,MDRXB550APB,наушник sony mdrxb550ap чёрный
4,MDRZX310L,наушник sony mdrzx310l синий
5,MDRZX310R,наушник sony mdrzx310r красный
6,MDREX15APB,наушник вкладыш sony mdrex15ap 12м чёрный пров...
7,MDREX15APLI,наушник вкладыш sony mdrex15lp 12м голубой про...
8,MDREX15LPB,наушник вкладыш sony mdrex15lp 12м чёрный пров...
9,MDRZX110B,наушник накладный sony mdrzx110 12м чёрный про...


## Векторизация

### TF-IDF

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Создаем объект TfidfVectorizer и обучаем его на текстах из колонки DESC
tfidf_vectorizer = TfidfVectorizer()
item_tfidf = tfidf_vectorizer.fit_transform(df['DESC'])

# Проверяем размерность матрицы
print(item_tfidf.shape)


(24517, 6481)


### Bag Of Words

In [38]:
from sklearn.feature_extraction.text import CountVectorizer

# Создаем объект CountVectorizer и обучаем его на текстах из колонки DESC
count_vectorizer = CountVectorizer()
item_bow = count_vectorizer.fit_transform(df['DESC'])

# Проверяем размерность матрицы
print(item_bow.shape)

(24517, 6481)


Будем использовать матрицу item_tfidf/item_bow как матрицу признаков X, а столбец ITEM исходного датафрейма как целевую переменную y

In [39]:
from sklearn.model_selection import train_test_split

# Выделяем целевую переменную
y = df['ITEM']

# Проверяем размерности матрицы признаков и целевой переменной
assert item_tfidf.shape[0] == len(y)

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(item_tfidf, y, test_size=0.2, random_state=42)


## Сравниваем классификаторы

### Decision tree

In [40]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Обучаем решающее дерево
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)

# Предсказываем значения на тестовой выборке
y_pred = dt_clf.predict(X_test)

# Оцениваем качество модели на тестовой выборке
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.7657014681892332


### Random Forest

In [41]:
from sklearn.ensemble import RandomForestClassifier

# Создание и обучение модели
rfmodel = RandomForestClassifier(random_state=42)
rfmodel.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = rfmodel.predict(X_test)

# Оценка точности
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.7975122349102773


### K-Nearest Neighbours

In [42]:
from sklearn.neighbors import KNeighborsClassifier

# Создание и обучение модели
knmodel = KNeighborsClassifier()
knmodel.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = knmodel.predict(X_test)

# Оценка точности
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.6829119086460033


### XGBoost

In [43]:
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

# Создаем экземпляр LabelEncoder
label_encoder = LabelEncoder()

# Преобразуем целевую переменную в числовые значения
y_encoded = label_encoder.fit_transform(y)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(item_tfidf, y_encoded, test_size=0.2, random_state=42)

# Создание и обучение модели
xgbmodel = XGBClassifier(random_state=42)
xgbmodel.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = xgbmodel.predict(X_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


## Тестируем на реальных данных

In [ ]:

test_df = pd.read_excel('C:\\Users\\223hy\\OneDrive\\Рабочий стол\\matcher\\sku_matcher\\test_df.xlsx')
test_df

,DESC
0,Наушники вставные MDR-E9LP Blue (731705)
1,Наушники вставные с микрофоном Sony MDR-EX15AP...
2,Наушники вставные MDR-E9LP Black (727692)
3,Наушники вставные MDR-E9LP Black (727692)
4,Наушники вставные MDR-E9LP Black (727692)
...,...
535,Наушники вставные c мик-ом Sony MDREX155AP bla...
536,Наушники вставные Sony MDR-EX15LPLIZ(PLIC)(AE)...
537,Наушники вставные MDR-E9LP Black (727692)
538,Наушники вставные MDR-E9LP Blue (731705)


In [ ]:
# применение функции очистки текста к колонке 'DESC'
test_df['DESC'] = test_df['DESC'].apply(clean_text)

# применение функции лемматизации к русским словам в колонке 'DESC'
# test_df['DESC'] = test_df['DESC'].apply(lemmatize)
test_df

,DESC
0,наушники вставные mdre9lp blue 731705
1,наушники вставные микрофоном sony mdrex15apvcz...
2,наушники вставные mdre9lp black 727692
3,наушники вставные mdre9lp black 727692
4,наушники вставные mdre9lp black 727692
...,...
535,наушники вставные c миком sony mdrex155ap blac...
536,наушники вставные sony mdrex15lplizplicae 9467...
537,наушники вставные mdre9lp black 727692
538,наушники вставные mdre9lp blue 731705


In [ ]:
# Векторизуем текст в колонке DESC в test_df
test_item_tfidf = tfidf_vectorizer.transform(test_df['DESC'].astype(str))

# Применяем обученную модель к test_item_tfidf и получаем вероятности для каждой метки класса
predicted_prob = xgbmodel.predict_proba(test_item_tfidf)

# Выбираем только те значения, для которых вероятность превышает порог
predicted_labels = [xgbmodel.classes_[np.argmax(pred)] if np.max(pred) > 0.9 else None for pred in predicted_prob]

# Записываем соответствующие значения MATERIAL SAP в новый столбец
test_df['PREDICTED_ITEM'] = predicted_labels


### Сохраняем распознанные данные

In [ ]:
test_df.to_excel('output_file.xlsx', index=False)